In [5]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
import numpy as np

In [6]:
data = pd.read_csv('survey_lung_cancer.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   GENDER                 309 non-null    object
 1   AGE                    309 non-null    int64 
 2   SMOKING                309 non-null    int64 
 3   YELLOW_FINGERS         309 non-null    int64 
 4   ANXIETY                309 non-null    int64 
 5   PEER_PRESSURE          309 non-null    int64 
 6   CHRONIC DISEASE        309 non-null    int64 
 7   FATIGUE                309 non-null    int64 
 8   ALLERGY                309 non-null    int64 
 9   WHEEZING               309 non-null    int64 
 10  ALCOHOL CONSUMING      309 non-null    int64 
 11  COUGHING               309 non-null    int64 
 12  SHORTNESS OF BREATH    309 non-null    int64 
 13  SWALLOWING DIFFICULTY  309 non-null    int64 
 14  CHEST PAIN             309 non-null    int64 
 15  LUNG_CANCER            

In [7]:
data = data.sample(frac=1).reset_index(drop=True)
data

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,F,56,2,2,2,2,2,1,2,2,2,1,1,2,2,YES
1,F,63,2,2,2,2,2,2,2,2,1,2,2,2,2,YES
2,F,68,2,1,2,1,1,2,1,1,1,1,1,1,1,NO
3,F,54,2,2,2,2,2,2,1,2,1,2,1,2,2,YES
4,M,61,1,1,2,2,2,1,1,1,2,1,1,1,1,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,F,59,1,2,2,2,2,2,2,2,1,2,2,2,1,YES
305,M,56,2,1,1,1,2,1,1,2,1,1,2,1,2,YES
306,F,70,1,1,1,2,2,1,2,1,2,2,2,1,1,YES
307,M,63,2,2,2,1,2,2,2,2,1,1,2,1,1,YES


In [8]:
def KNN(data, k):
    accuracy_table = []
    mean = 0
    std = 0
    for i in range(0, 500):
        data = data.sample(frac=1).reset_index(drop=True)
        X = data.iloc[:, 1:-1]
        y = data.iloc[:, -1]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        knn_classifier = KNeighborsClassifier(n_neighbors=k)
        knn_classifier.fit(X_train, y_train)
        y_pred = knn_classifier.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred) * 100
        accuracy_table.append(accuracy)
    mean = round(np.mean(accuracy_table), 2)
    std = round(np.std(accuracy_table), 2)
    print("Dokładność klasyfikatora KNN dla k = " + str(k) + " wynosi: " + str(mean) + "% a odchylenie standardowe: " + str(std))

In [9]:
def NaiveBayes(data):
    accuracy_table = []
    mean = 0
    std = 0
    for i in range(0, 500):
        X = data.iloc[:, 1:-1]
        y = data.iloc[:, -1]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        naive_bayes_classifier = GaussianNB()
        naive_bayes_classifier.fit(X_train, y_train)
        y_pred = naive_bayes_classifier.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred) * 100
        accuracy_table.append(accuracy)
    mean = round(np.mean(accuracy_table), 2)
    std = round(np.std(accuracy_table), 2)
    print("Dokładność naiwnego klasyfikatora bayesa wynosi = " + str(mean) + "% a odchylenie standardowe: " + str(std))

In [10]:
for i in range(2, 11):
    KNN(data, i)

Dokładność klasyfikatora KNN dla k = 2 wynosi: 83.52% a odchylenie standardowe: 4.36
Dokładność klasyfikatora KNN dla k = 3 wynosi: 88.08% a odchylenie standardowe: 3.88
Dokładność klasyfikatora KNN dla k = 4 wynosi: 87.08% a odchylenie standardowe: 4.05
Dokładność klasyfikatora KNN dla k = 5 wynosi: 87.68% a odchylenie standardowe: 3.67
Dokładność klasyfikatora KNN dla k = 6 wynosi: 87.65% a odchylenie standardowe: 3.75
Dokładność klasyfikatora KNN dla k = 7 wynosi: 87.63% a odchylenie standardowe: 3.83
Dokładność klasyfikatora KNN dla k = 8 wynosi: 87.54% a odchylenie standardowe: 3.99
Dokładność klasyfikatora KNN dla k = 9 wynosi: 87.06% a odchylenie standardowe: 4.04
Dokładność klasyfikatora KNN dla k = 10 wynosi: 87.35% a odchylenie standardowe: 3.93


In [11]:
NaiveBayes(data)

Dokładność naiwnego klasyfikatora bayesa wynosi = 88.71% a odchylenie standardowe: 0.0
